
# Fader Network - Démonstration

Ce notebook permet de reproduire l'intégralité du code et des expériences décrites dans le dépôt **Fader Network**.
Vous trouverez :
1. La configuration des dépendances
2. La préparation des datasets
3. Le chargement des modèles (autoencodeur et discriminateur)
4. La visualisation des données
5. L'évaluation et la prédiction

---


In [ ]:

# Importation des bibliothèques nécessaires
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from model import AutoEncoder, Discriminator, CelebADataset

# Configuration du dispositif GPU/CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")



## Chargement des datasets
Nous utilisons les images CelebA et leurs attributs pour créer trois datasets : entraînement, validation et test.


In [ ]:

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Répertoires de données
img_dir = "datas/img_align_celeba/img_align_celeba/"
attr_path = "datas/list_attr_celeba.csv"
partition_path = "datas/list_eval_partition.csv"

# Chargement des datasets
batch_size = 32

dataset_train = CelebADataset(img_dir, attr_path, partition_path, transform, split="train")
dataset_val = CelebADataset(img_dir, attr_path, partition_path, transform, split="valid")
dataset_test = CelebADataset(img_dir, attr_path, partition_path, transform, split="test")

print(f"Nombre d'images dans le dataset d'entraînement : {len(dataset_train)}")
print(f"Nombre d'images dans le dataset de validation : {len(dataset_val)}")
print(f"Nombre d'images dans le dataset de test : {len(dataset_test)}")

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)



## Chargement des modèles
Nous chargeons l'autoencodeur et le discriminateur avec leurs poids sauvegardés.


In [ ]:

# Chargement des modèles
autoencoder = AutoEncoder().to(device)
autoencoder.load_state_dict(torch.load("sexe/train_sexe70epoch/autoencoder.pth", map_location=device))
autoencoder.eval()

discriminator = Discriminator().to(device)
discriminator.load_state_dict(torch.load("sexe/train_sexe70epoch/discriminator.pth", map_location=device))
discriminator.eval()



## Visualisation d'une image avec ses attributs


In [ ]:

def affiche_image_et_attributs(dataset, idx):
    image, attributs = dataset[idx]
    attribute_names = dataset.get_attribute_names()

    plt.imshow(image.permute(1, 2, 0).numpy())
    plt.axis('off')
    plt.show()

    print("Attributs associés à l'image :")
    for i, attr in enumerate(attributs):
        print(f"{attribute_names[i]} : {'Oui' if attr.item() == 1 else 'Non'}")

# Affichage d'une image du dataset de validation
affiche_image_et_attributs(dataset_val, 10)



## Évaluation du modèle sur le dataset de test


In [ ]:

criterion = nn.MSELoss()

def evaluer_autoencodeur(dataloader):
    ae_loss_total = 0
    disc_loss_total = 0
    num_batches = 0

    for batch_images, batch_attrs in tqdm(dataloader):
        batch_images = batch_images.to(device)
        batch_attrs = batch_attrs.to(device)

        with torch.no_grad():
            z = autoencoder.encoder(batch_images)
            reconstructed_images = autoencoder.decoder(z, batch_attrs)
            ae_loss = criterion(reconstructed_images, batch_images)

            disc_pred_real = discriminator(batch_images)
            disc_pred_fake = discriminator(reconstructed_images)
            disc_loss = 0.5 * (criterion(disc_pred_real, torch.ones_like(disc_pred_real)) +
                               criterion(disc_pred_fake, torch.zeros_like(disc_pred_fake)))

            ae_loss_total += ae_loss.item()
            disc_loss_total += disc_loss.item()
            num_batches += 1

    ae_loss_avg = ae_loss_total / num_batches
    disc_loss_avg = disc_loss_total / num_batches

    print(f"Perte moyenne de l'autoencodeur sur le dataset de test : {ae_loss_avg:.4f}")
    print(f"Perte moyenne du discriminateur sur le dataset de test : {disc_loss_avg:.4f}")

# Évaluation du modèle sur le dataset de test
evaluer_autoencodeur(dataloader_test)



## Prédiction sur une image spécifique


In [ ]:

def predit_image(image_path):
    pil_img = Image.open(image_path).convert("RGB")
    plt.imshow(pil_img)
    plt.axis('off')
    plt.show()

    input_tensor = transform(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        z = autoencoder.encoder(input_tensor)
        output_tensor = autoencoder.decoder(z, torch.ones((1, 1), device=device))

    output_image = output_tensor.squeeze(0).cpu().permute(1, 2, 0)
    plt.imshow(output_image.numpy())
    plt.axis('off')
    plt.show()

# Prédiction sur une image spécifique
predit_image("datas/img_align_celeba/img_align_celeba/000001.jpg")
